In [4]:

!pip install torch torchvision
!pip install Pillow
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.4 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-_zreh2bv
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-_zreh2bv
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369489 sha256=58921510e3eba3b339963c6463ce39b475f8f4512152e6fa6587de6efd243f4a
  Stored in directory: /tmp/pip-ephem-wheel-cache-plq3lip6/wheels/3f/7c/a4/9b490845988bf7a4db33674d52f709f088f64392063872eb9a
Successfully built clip


In [5]:
import torch
import clip
from PIL import Image
import requests
from io import BytesIO

In [6]:
def get_caption_from_file(image_path, candidate_file, model_name="ViT-B/32"):
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Load the CLIP model and its preprocessing function.
    model, preprocess = clip.load(model_name, device=device)

    # Load and preprocess the image.
    image = Image.open(image_path)
    image_input = preprocess(image).unsqueeze(0).to(device)

    # Read candidate captions from file (one caption per line).
    with open(candidate_file, "r", encoding="utf-8") as f:
        candidate_captions = [line.strip() for line in f if line.strip()]

   # Tokenize the candidate captions.
    text_inputs = clip.tokenize(candidate_captions).to(device)

    # Compute the image and text features.
    with torch.no_grad():
        image_features = model.encode_image(image_input)
        text_features = model.encode_text(text_inputs)

    # Normalize the features to get cosine similarity as the dot product.
    image_features = image_features / image_features.norm(dim=-1, keepdim=True)
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)

    # Compute cosine similarity between the image and each candidate caption.
    similarities = (image_features @ text_features.T).squeeze(0)

    # Select the caption with the highest similarity.
    best_caption_index = torch.argmax(similarities).item()
    best_caption = candidate_captions[best_caption_index]

    return best_caption


In [7]:
def chat():
    image_url = input("Please enter the image URL: ").strip()
    try:
        response = requests.get(image_url)
        response.raise_for_status()
        image_data = BytesIO(response.content)
        return image_data
    except Exception as e:
        print("Failed to retrieve the image. Error:", e)
        exit(1)

In [8]:
if __name__ == "__main__":
  while(True):
    image_source = chat()
    candidate_file = "candidate_captions.txt"

    caption = get_caption_from_file(image_source, candidate_file)
    print("Selected Caption:", caption)

    print("Would you like to continue? (y/n)")
    choice = input().strip().lower()
    if choice != 'y':
        print("Bye 👋")
        break


Please enter the image URL: https://www.google.com/url?sa=i&url=https%3A%2F%2Fstock.adobe.com%2Fsearch%3Fk%3Ddogs%2Bplaying&psig=AOvVaw2rndL9WBo8Wfq7s4JSxvL7&ust=1739592493834000&source=images&cd=vfe&opi=89978449&ved=0CBQQjRxqFwoTCPja8palwosDFQAAAAAdAAAAABAE


100%|███████████████████████████████████████| 338M/338M [00:07<00:00, 50.0MiB/s]


UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x7c3b9f994720>

In [9]:
def get_caption_from_file(image_data, candidate_file, model_name="ViT-B/32"):
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Load the CLIP model and its preprocessing function.
    model, preprocess = clip.load(model_name, device=device)

    # Load and preprocess the image.
    # The change is here. We now open the image from the BytesIO object directly
    image = Image.open(image_data)
    image_input = preprocess(image).unsqueeze(0).to(device)

    # Read candidate captions from file (one caption per line).
    with open(candidate_file, "r", encoding="utf-8") as f:
        candidate_captions = [line.strip() for line in f if line.strip()]

    # Tokenize the candidate captions.
    text_inputs = clip.tokenize(candidate_captions).to(device)

 # Compute the image and text features.
    with torch.no_grad():
        image_features = model.encode_image(image_input)
        text_features = model.encode_text(text_inputs)

    # Normalize the features to get cosine similarity as the dot product.
    image_features = image_features / image_features.norm(dim=-1, keepdim=True)
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)

    # Compute cosine similarity between the image and each candidate caption.
    similarities = (image_features @ text_features.T).squeeze(0)

    # Select the caption with the highest similarity.
    best_caption_index = torch.argmax(similarities).item()
    best_caption = candidate_captions[best_caption_index]

    return best_caption


In [12]:
!pip install torch torchvision transformers pillow
!pip install git+https://github.com/openai/CLIP.git


  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-s2ab86qz
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-s2ab86qz
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


In [15]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

image = preprocess(Image.open("/content/images (1).jpg")).unsqueeze(0).to(device)

captions = [
    "A dog playing in the park.",
    "A cat sitting on the sofa.",
    "A person riding a bicycle.",
    "A beautiful sunset over the mountains."
]

text = clip.tokenize(captions).to(device)

with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)

    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    similarity = (image_features @ text_features.T).squeeze(0)

best_caption = captions[similarity.argmax().item()]
print("Predicted Caption:", best_caption)

Predicted Caption: A cat sitting on the sofa.
